### sequencial problem

# Architecture of RNN

|Hyperperameter/ Layer Type|What does it do|
|--------------------------|---------------|
|Input text|Target text sequence you like to discover pattern in|
|Input Layer|Takes the target seqyence
|Text Vectorization Layer|Map input sequence to number|
|Embedding|Turn mapping of the text vactor to embedding matrix|
|RNN Cell|Find pattern in the sequence|
|Hidden activation|Adds non-linearity to the learnend features|
|Pooling layer|Reduce the dimensionality of learned sequence features|
|Fully connected layer|Further refine learning features from recurrent layers|
|Output Layer|take learning features and output them in shape of target labels|
|Output Activation|Adds non-linarity to the output layers|

# Get text data sets

In [1]:
! wget"https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb"

The filename, directory name, or volume label syntax is incorrect.


In [2]:
# download the data
import urllib.request
import zipfile

url = "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
filename = "NLP_dataset.zip"

# Download the file
print(f"Downloading {filename}...")
urllib.request.urlretrieve(url, filename)
print("Download complete.")

# unzip the downloaded file
zip_ref = zipfile.ZipFile("NLP_dataset.zip")
zip_ref.extractall()
zip_ref.close()

Download complete.


## visualizing the Data

In [3]:
import pandas as pd
training_data = pd.read_csv("NLP_train.csv")
test_data = pd.read_csv("NLP_test.csv")
training_data.shape,test_data.shape

((7613, 5), (3263, 4))

In [4]:
training_data.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
training_data["target"].unique()

array([1, 0], dtype=int64)

## shuffle the data

In [6]:
training_data = training_data.sample(frac=1,random_state=42)
training_data.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


## checking the data Balancing

In [7]:
training_data.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

## creating a validation datasets

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(training_data["text"].to_numpy(),
                                                  training_data["target"].to_numpy(),
                                                  test_size=0.1,
                                                  random_state=42)
print(f"x_train ={x_train.size}"),print(f"x_valid ={x_valid.size}"),print(f"y_train ={y_train.size}"),print(f"y_valid ={y_valid.size}")

x_train =6851
x_valid =762
y_train =6851
y_valid =762


(None, None, None, None)

In [9]:
x_train[:10],y_train[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Tokenization vs Embedding

* **Tokenization**:stright mapping from token to number (can be modelled but quickly get's to big)
    * **word-level Tokenization**
    * **Charactors level Tokenization**
    * **sub-words level Tokenization**
* **Embedding**: Richer representation of relationships between tokens (can limit size and can be learned)
    * **Create your own Embedding**
    * **Reuse a pre train Embedding**

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization 

In [11]:
text_vectorizer = TextVectorization(max_tokens=None,# the maximum zize of the vocabulary automaticaly words <00v>
                                    standardize="lower_and_strip_punctuation", # technique for standardization
                                    split="whitespace", # how to split the tokens from words
                                    ngrams=None, # create a group of words to tokenize(none => one word==one token)
                                    output_mode="int", # how to map token to different data type
                                    output_sequence_length=None, # make the length of every row of data same (choose mac length = largest peragraph)
                                    pad_to_max_tokens=False # pedd the small words with 0's to make then same size
                                   )

In [12]:
# fing the avrage length of the peragraphs in our data
round(sum([len(i.split()) for i in x_train])/len(x_train))

15

In [13]:
# setup the veciorization veriables
MAX_VOCAB_LENGTH = 10000 # max number of words you have in your vocabulary
MAX_LENGTH = 15 #MAX LENGTH OF UOR SEQUENCE WILL BE

text_vectorizer = TextVectorization(max_tokens=MAX_VOCAB_LENGTH,
                                   output_mode="int",
                                   output_sequence_length=MAX_LENGTH)

In [14]:
# fitting the text vector to the traning text
text_vectorizer.adapt(x_train)

In [15]:
# checing with the example
import random
sample_sentence=random.choice(x_train)
print(sample_sentence)
text_vectorizer([sample_sentence])

@BLutz10 But the rioting began prior to the decision for the indictment so you're not really making sense at this pointÛ_


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[6083,   30,    2,  367, 1550, 2782,    5,    2, 3067,   10,    2,
           1,   28,  172,   34]], dtype=int64)>

In [16]:
#  getting the information in our vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top5_words_in_vocab = words_in_vocab[:5]
bottom5_words_in_vocab = words_in_vocab[-5:]

print(f"number of words ={len(words_in_vocab)}")
print(f"top 5 words: {top5_words_in_vocab}")
print(f"bottom 5 words: {bottom5_words_in_vocab}")

number of words =10000
top 5 words: ['', '[UNK]', 'the', 'a', 'in']
bottom 5 words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Creating a Embedding useing Embbdeing Layer

In [17]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=MAX_VOCAB_LENGTH,
                            output_dim=128,)

In [18]:
# checing with the example
import random
sample_sentence=random.choice(x_train)
print(sample_sentence)
embedding(text_vectorizer([sample_sentence]))

Wildfire Burns On California U.s. China Economic Net Û_ : http://t.co/U2dO2mC2ri  http://t.co/3oM3xw6CZ8


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01280217,  0.03925582,  0.00553781, ...,  0.04052489,
         -0.02757215,  0.02086021],
        [ 0.020749  ,  0.01384989,  0.0394888 , ...,  0.03879805,
          0.00669261, -0.02248528],
        [ 0.02057335,  0.045073  , -0.02360734, ...,  0.04206521,
          0.00200303,  0.0016489 ],
        ...,
        [ 0.01456434, -0.04610474, -0.02598318, ..., -0.01655135,
         -0.03161448, -0.03317741],
        [ 0.01456434, -0.04610474, -0.02598318, ..., -0.01655135,
         -0.03161448, -0.03317741],
        [ 0.01456434, -0.04610474, -0.02598318, ..., -0.01655135,
         -0.03161448, -0.03317741]]], dtype=float32)>

## Experimant's we're running

|Experiment Number|Model|
|-----------------|-----|
|1|Naive Bayes with TF-DIF encoder(baseline)|
|2|Feed forword neural network(dence model)|
|3|LSTM (RNN)|
|4|GRU (RNN)|
|5|Bidirectional-LSTM (RNN)|
|6|1D Convolutional Nural Network|
|7|Pretrain Feature Extractor|
|8|Pretrain Feature Extractor (10% data)|

_**TfidfVectorizer** is a powerful tool in Natural Language Processing (NLP) provided by the scikit-learn library. Its primary function is to convert a collection of raw text documents into a matrix of TF-IDF features. This numerical representation of text is essential for many machine learning algorithms, which typically require numerical input._

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create tokenization and modeling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convet words into numbers useing tfidf
    ("clf",MultinomialNB()) #model to test
])

# fit the pipeline to the train data
model_0.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [20]:
# Evaluate the model
baseline_score = model_0.score(x_valid,y_valid)
baseline_score

0.7926509186351706

In [21]:
# make predictions function
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision, model_recall, model_fscore, _ = precision_recall_fscore_support(y_true,y_pred,average="binary")

    model_result={"accuracy":model_accuracy,
                 "precision":model_precision,
                 "recall":model_recall,
                 "f1":model_fscore}
    return model_result

In [22]:
# get baseline model result
M1_results = calculate_results(y_train,
                              model_0.predict(x_train))
M1_results

{'accuracy': 88.71697562399649,
 'precision': 0.9629629629629629,
 'recall': 0.7649674991447143,
 'f1': 0.8526215443279314}

In [23]:
# build an model with sample API
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)

x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
output=layers.Dense(1,activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs,output,name="model_2_dence")

In [24]:
model_2.summary()

Model: "model_2_dence"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model_2.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [26]:
mode1_2_history = model_2.fit(x_train,
                             y_train,
                             epochs=5,
                             validation_data=(x_valid,y_valid))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.6891 - loss: 0.6107 - val_accuracy: 0.7612 - val_loss: 0.5336
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8136 - loss: 0.4447 - val_accuracy: 0.7848 - val_loss: 0.4757
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8581 - loss: 0.3512 - val_accuracy: 0.7927 - val_loss: 0.4566
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8880 - loss: 0.2896 - val_accuracy: 0.7927 - val_loss: 0.4669
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9092 - loss: 0.2421 - val_accuracy: 0.7782 - val_loss: 0.4764


In [27]:
model_2.evaluate(x_valid, y_valid)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7782 - loss: 0.4764 


[0.4763517677783966, 0.778215229511261]

In [28]:
model_2.predict(x_train)

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[0.09887896],
       [0.07422978],
       [0.94395447],
       ...,
       [0.73381627],
       [0.8843655 ],
       [0.03024209]], dtype=float32)

In [29]:
model_2_pred = tf.squeeze(tf.round(model_2.predict(x_train)))
model_2_pred

215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


<tf.Tensor: shape=(6851,), dtype=float32, numpy=array([0., 0., 1., ..., 1., 1., 0.], dtype=float32)>

In [30]:
# get baseline model result
M2_results = calculate_results(y_train,
                              model_2_pred)
M2_results

{'accuracy': 93.38782659465772,
 'precision': 0.9563932002956393,
 'recall': 0.8853917208347588,
 'f1': 0.9195238941197371}

## Visualizing the models layers

In [31]:
model_2.summary()

Model: "model_2_dence"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [33]:
# get the embedding layer
embed_weight = model_2.get_layer("embedding").get_weights()
embed_weight

[array([[ 0.00385029, -0.06833327, -0.01388544, ..., -0.02162649,
         -0.05241815, -0.01448281],
        [-0.00223897,  0.01509984,  0.03058731, ...,  0.0088402 ,
         -0.02771979, -0.03795448],
        [-0.04277845, -0.03196308,  0.00032645, ..., -0.01240611,
         -0.05236866, -0.00413881],
        ...,
        [ 0.03872749, -0.02471119, -0.00502457, ...,  0.0057093 ,
          0.02998267,  0.04538938],
        [-0.06774551, -0.0585929 ,  0.02884754, ..., -0.08509322,
          0.00476634,  0.03632554],
        [-0.1149267 , -0.09836286,  0.0905294 , ..., -0.06761543,
         -0.09628862,  0.04649833]], dtype=float32)]

# Recurrent neural Network RNN